Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
![CC BY License Logo](https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png)
___

# Latent Dirichlet Allocation (LDA) Topic Modeling

**Description of methods in this notebook:**
This [notebook](https://docs.tdm-pilot.org/key-terms/#jupyter-notebook) demonstrates how to do topic modeling on a [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor) and/or [Portico](https://docs.tdm-pilot.org/key-terms/#portico) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) using [Python](https://docs.tdm-pilot.org/key-terms/#python). The following processes are described:

* Importing your [dataset](https://docs.tdm-pilot.org/key-terms/#dataset)
* Importing libraries including `gensim`, `nltk`, and `pyLDAvis`
* Writing a helper function to help clean up a single [token](https://docs.tdm-pilot.org/key-terms/#token)
* Building a gensim dictionary and training the model
* Computing a topic list
* Visualizing the topic list

**Difficulty:** Intermediate

**Knowledge Required:** 
* [Python Basics I](./0-python-basics-1.ipynb)
* [Python Basics II](./0-python-basics-2.ipynb)
* [Python Basics III](./0-python-basics-3.ipynb)

**Knowledge Recommended:**
* [Exploring Metadata](./1-metadata.ipynb)
* A familiarity with [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) is helpful but not required.

**Completion time:** 90 minutes

**Data Format:** [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor)/[Portico](https://docs.tdm-pilot.org/key-terms/#portico) [JSON Lines (.jsonl)](https://docs.tdm-pilot.org/key-terms/#jsonl)

**Libraries Used:**
* **[json](https://docs.tdm-pilot.org/key-terms/#json-python-library)** to convert our dataset from json lines format to a Python list
* **[gensim](https://docs.tdm-pilot.org/key-terms/#gensim)** to accomplish the topic modeling
* **[NLTK](https://docs.tdm-pilot.org/key-terms/#nltk)** to help [clean](https://docs.tdm-pilot.org/key-terms/#clean-data) up our dataset
* **pyldavis** to visualize our topic model
____

## Import your dataset

You'll use the tdm_client library to automatically upload your dataset. We import the `Dataset` module from the `tdm_client` library. The tdm_client library contains functions for connecting to the JSTOR server containing our [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset). To analyze your dataset, use the [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) provided when you created your [dataset](https://docs.tdm-pilot.org/key-terms//#dataset). A copy of your [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) was sent to your email when you created your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus). It should look like a long series of characters surrounded by dashes. If you haven't created a dataset, feel free to use a sample dataset. Here's a [list by discipline](https://docs.tdm-pilot.org/sample-datasets/). Advanced users can also [upload a dataset from their local machine](https://docs.tdm-pilot.org/uploading-a-dataset/).

In [1]:
#Importing your dataset with a dataset ID
import tdm_client
#Load the sample dataset, the full run of Shakespeare Quarterly from 1950-2013.
tdm_client.get_dataset("f6ae29d4-3a70-36ee-d601-20a8c0311273", "sampleJournalAnalysis") 

'datasets/sampleJournalAnalysis.jsonl'

Initialize a dataset object. 

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the documents in the dataset and build a list of doucments where each document is a list of tokens. For demonstration purposes, we are going to limit to 25 documents in the dataset. 

In [3]:
import json

documents = []
doc_count = 0
# Limit the number of documents, set to None to not limit.
limit_to = 25

with open("./datasets/sampleJournalAnalysis.jsonl") as input_file:
    for line in input_file:
        doc = json.loads(line)
        unigram_count = doc["unigramCount"]
        document_tokens = []
        for token, count in unigram_count.items():
            clean_token = process_token(token)
            if clean_token is None:
                continue
            document_tokens += [clean_token] * count
        documents.append(document_tokens)
        doc_count += 1 
        if (limit_to is not None) and (doc_count >= limit_to):
            break
                

Build a gensim dictionary corpus and then train the model.

In [4]:
import gensim

num_topics = 10

dictionary = gensim.corpora.Dictionary(documents)

# Remove terms that appear in less than 10% of documents and more than 75% of documents.
dictionary.filter_extremes(no_below=doc_count * .10, no_above=0.75)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train the LDA model.
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=15
)


Print the most significant terms, as determined by the model, for each topic.

In [5]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))

Topic 0     stage theatre lear john like folger booth effect text prospero
Topic 1     sexual macbeth book gloss seem definition tragedy booth language idea
Topic 2     like lear form bassanio king made make stage shylock work
Topic 3     make book performance account privy much theatre note order evidence
Topic 4     portrait much john date production william note presented original state
Topic 5     upon juliet parallel book richard moment original despair like make
Topic 6     macbeth character stage critical could dryden line made like work
Topic 7     bassanio shylock merchant money make male like woman venice love
Topic 8     king lear edgar form like report book word made world
Topic 9     henry william john richard editor mean gentleman manuscript used sonnet


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/). This visualization takes several minutes to an hour to generate depending on the size of your dataset. To run, remove the `#` symbol on the line below and run the cell. 

In [6]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.010347  0.057965       1        1  24.112938
4      0.084406 -0.041413       2        1  22.150112
7     -0.110049 -0.074801       3        1  20.646769
6      0.027494  0.074314       4        1  14.741884
9      0.143811 -0.113890       5        1   6.863354
0      0.088562  0.138281       6        1   5.671492
1     -0.110538  0.034562       7        1   2.657337
5     -0.081940 -0.003873       8        1   1.795799
3     -0.023507 -0.057261       9        1   1.344468
2     -0.007891 -0.013884      10        1   0.015850, topic_info=          Term       Freq      Total Category  logprob  loglift
1235  portrait  68.000000  68.000000  Default  30.0000  30.0000
1496   macbeth  49.000000  49.000000  Default  29.0000  29.0000
12        book  53.000000  53.000000  Default  28.0000  28.0000
268       lear  79.000000  79.000000  Default  27.0000  27.0000
805   bassanio  44.000000  44.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
264       john   0.001638  78.302376  Topic10  -7.3281  -2.0253
969   merchant   0.001613  37.751129  Topic10  -7.3434  -1.3110
94        much   0.001620  57.051991  Topic10  -7.3389  -1.7195
150    theatre   0.001614  53.034378  Topic10  -7.3424  -1.6500
279       mean   0.001612  38.787880  Topic10  -7.3438  -1.3386

[619 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
164       1  0.611486  abandoned
164       8  0.305743  abandoned
165       1  0.254646       able
165       2  0.254646       able
165       3  0.084882       able
...     ...       ...        ...
400       5  0.023728      world
400       6  0.023728      world
1130      1  0.825519      worst
1130      3  0.097120      worst
1130      4  0.048560      worst

[1579 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 8, 7, 10, 1, 2, 6, 4, 3])